###import packages

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np
import pickle

/Users/timklabjan/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


###read in data

In [2]:
data = pd.read_csv("content_topic_prediction/Topic_prediction_data_train.txt",sep='\t',header=None)
data.columns = ['text','uid','type','timestamp']
labels = pd.read_csv("content_topic_prediction/Topic_prediction_labels_train.txt",sep='\t',header=None)
labels.columns = ['label']
data['label'] = labels['label']

In [3]:
data.head()

,text,uid,type,timestamp,label
0,distinct consubstanti translat roman missal pu...,0,Q,0.01,9
1,cathol christian power bind loos catholic beli...,0,Q,0.01,0
2,read scriptur common worship christian church ...,0,Q,0.01,9
3,accord person polit libertarian compat golden ...,0,Q,0.01,0
4,criteria cathol determin testament charact pre...,0,Q,0.01,0


###drop null values

In [4]:
data = data[(data['text'].notnull())]

###convert text data to term frequency vectors

In [11]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000,ngram_range=(1,1))
train_data_features = vectorizer.fit_transform(data["text"])
pickle.dump(vectorizer,open("vectorizer.p","wb" ))
features= np.array(train_data_features)

###tune model paramaters

In [8]:
#lr = LogisticRegression(random_state=1)
#lr_params = {'penalty':('l1','l2'),'C':[0.1,1,10],'fit_intercept':(True,False)}
#lr_cv = GridSearchCV(lr,lr_params,cv=2)
#nb = MultinomialNB()
#nb_params = {'alpha':[0.001,0.5,1]}
#nb_cv = GridSearchCV(nb,nb_params,cv=2)
#svc = LinearSVC(random_state=1)
#svc_params = {'penalty':('l1','l2'),'C':[0.1,1,10]}
#svc_cv = GridSearchCV(lr,lr_params,cv=2)

#lr_cv.fit(train_data_features,data["label"])
#nb_cv.fit(train_data_features,data["label"])
#svc_cv.fit(train_data_features,data["label"])
#print(lr_cv.best_params_)
#print(nb_cv.best_params_)
#print(svc_cv.best_params_)

###train ensemble model, get cross validation accuracy scores

In [18]:
#classifier = VotingClassifier(estimators=[('lr',LogisticRegression(random_state=1,C=1,penalty='l2')),('nb',MultinomialNB(alpha=0.001)),('svc',LinearSVC(random_state=1,C=1,penalty='l2'))],voting='hard')
classifier = VotingClassifier(estimators=[('lr',LogisticRegression()),('nb',MultinomialNB()),('svc',LinearSVC())],voting='hard')
scores = cross_val_score(classifier, train_data_features, data["label"], cv=5, scoring='accuracy')
#pickle.dump(classifier,open("ensemble_model.p","wb"))

###print average accuracy

In [17]:
scores.mean()
#tf and logreg - .7202
#tfidf and logreg - .709
#tf and ensemble - .7512
#tfidf and ensemble - .7286
#tf and tuned ensemble - .751
#tf and ensemble and bi/tri grams - 0.7344

0.7511493578555837

In [ ]:
classifier = pickle.load(open("ensemble_model.p","rb"))
preds = classifier.predict(train_data_features)
confusion_matrix(data["label"],preds)